In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import time
import plotly.io as pio
import plotly.graph_objs as go
import plotly.express as px
pio.renderers.default = 'notebook'

class StockInfo():
    def __init__(self,filename,t='y'):
        if t not in ['y','q']:
            print('Select "q" for quarterly || "y" for yearly.')
            self.state=0
            return
        if t == 'q':
            self.t = 'quarterly'
        else:
            self.t = 'yearly'
        try:
            with open(filename,'r') as file:
                self.stocks = [symbol.strip() for symbol in file.readlines()]
                if len(self.stocks) > 0:
                    self.get_data()
                    self.state = 1
                else:
                    self.state=0
        except Exception as e:
            print(f'✖ {filename} not found: {e}.')
            self.state = 0
            return
    
    def get_data(self):
        delete = []
        for stock in self.stocks:
            try:
                pd.read_csv(f'{stock}_financials-{self.t}.csv')
                print(f'Financials for {stock} already in cache.')
            except FileNotFoundError:
                try:
                    print(f'\nFetching financials for {stock}...')
                    ticker = yf.Ticker(stock)
                    cashflow_statement = ticker.get_cashflow(freq=self.t).transpose()
                    freecashflow_df = cashflow_statement['FreeCashFlow'].dropna(axis=0)
                    income_statement = ticker.get_incomestmt(freq=self.t).transpose()
                    financials_df = income_statement[['TotalRevenue','NetIncome']].dropna(axis=0)
                    if self.t == 'quarterly':
                        earnings_df = ticker.get_earnings_history()[['epsActual','epsEstimate']].iloc[::-1]
                        merged_df = pd.concat([freecashflow_df,financials_df,earnings_df],join='inner',axis=1).iloc[::-1]
                    else:
                        merged_df = pd.concat([freecashflow_df,financials_df],join='inner',axis=1).iloc[::-1]
                    merged_df.to_csv(f'{stock}_financials-{self.t}.csv')
                    print(f'✔ {stock} financials saved to cache.\n')
                    time.sleep(0.2)
                except Exception as e:
                    delete.append(stock)
                    print(f'✖ Could not fetch financials: {e}.')
                    print('continuing...\n')
                    continue
        for stock in delete:
            self.stocks.remove(stock)
                    
    def get_industry(self):
        self.sectors = pd.Series(dtype='object')
        for stock in self.stocks:
            try:
                print(f'\nFetching Industry of {stock}...')
                ticker = yf.Ticker(stock)
                self.sectors.loc[stock] = ticker.get_info()['industry']
                print(f'✔ Industry of {stock} loaded.')
                time.sleep(0.2)
            except Exception as e:
                print(f'✖ Could not fetch the industry for {stock}: {e}.')
                print('continuing...')
                continue
                
    def plot_earnings(self):
        if self.state == 0:
            return
        if self.t == 'yearly':
            print('\n✖ Earnings only quarterly available.')
            return
        for stock in self.stocks:
            df = pd.read_csv(f'{stock}_financials-{self.t}.csv',index_col=0,parse_dates=True)
            if self.t == 'quarterly':
                df.index = df.index.to_period('Q').astype('str')
            else:
                df.index = df.index.year.astype('str')
            for row,data in df.iterrows():
                if data['epsActual'] > data['epsEstimate']:
                    df.loc[row,'colorA'] = 'lime'
                elif data['epsActual'] == data['epsEstimate']:
                    df.loc[row,'colorA'] = 'royalblue'
                else:
                    df.loc[row,'colorA'] = 'red'
            
            fig = go.Figure()
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df['epsEstimate'],
                    name='Estimate',
                    mode='markers',
                    marker=dict(size=24,color='#000',line=dict(color='#888',width=1))
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df['epsActual'],
                    name='Actual',
                    mode='markers',
                    marker=dict(size=24,color=df['colorA'])
                )
            )
            fig.update_layout(
                paper_bgcolor='#000',
                plot_bgcolor='#000',
                xaxis=dict(gridcolor='#222'),
                yaxis=dict(gridcolor='#111'),
                font=dict(color='#fff'),
                title=f'{stock} quarterly EPS',
                showlegend=False
            )
            fig.show()
    
    def plot_income(self):
        if self.state == 0:
            return
        for stock in self.stocks:
            df = pd.read_csv(f'{stock}_financials-{self.t}.csv',index_col=0,parse_dates=True)
            if self.t == 'quarterly':
                df.index = df.index.to_period('Q').astype('str')
            else:
                df.index = df.index.year.astype('str')
            if df['NetIncome'].iloc[-1] <= 0 or df['NetIncome'].iloc[0] <= 0:
                cagr = 0
            else:
                cagr = round((df['NetIncome'].iloc[-1]/df['NetIncome'].iloc[0]-1)*100,2)
            mean = round((df['NetIncome'].pct_change()*100).dropna().mean(),2)
            std = round((df['NetIncome'].pct_change()*100).dropna().std(),2)
            fig = go.Figure()
            fig.add_trace(
                go.Bar(
                    x=df.index,
                    y=df['TotalRevenue'],
                    marker=dict(color='dodgerblue',line=dict(color='#000',width=1)),
                    name='Revenue'
                )
            )
            fig.add_trace(
                go.Bar(
                    x=df.index,
                    y=df['NetIncome'],
                    marker=dict(color='cyan',line=dict(color='#000',width=1)),
                    name='NetIncome'
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df['NetIncome'],
                    mode='lines+markers',
                    line=dict(color='coral',width=2),
                    marker=dict(size=8,color='coral',line=dict(color='#000',width=1)),
                    name='NetIncome'
                )
            )
            fig.update_layout(
                paper_bgcolor='#000',
                plot_bgcolor='#222',
                xaxis=dict(gridcolor='#333'),
                yaxis=dict(gridcolor='#333'),
                font=dict(color='#fff'),
                title=f'{stock} {self.t} Income',
                showlegend=False
            )
            fig.show()
            if cagr == 0:
                print(f'{stock} CAGR: NaN.')
            else:
                print(f'{stock} CAGR: {cagr:.2f}%.')
            print(f'Average {self.t} Growth: {mean:.2f}%')
            print(f'NetMargin Volatility: {std:.2f}%')
    
    def plot_trend(self):
        if self.state == 0:
            return
        for stock in self.stocks:
            df = pd.read_csv(f'{stock}_financials-{self.t}.csv',index_col=0,parse_dates=True)
            if self.t == 'quarterly':
                df.index = df.index.to_period('Q').astype('str')
            else:
                df.index = df.index.year.astype('str')
            df = (df[['TotalRevenue','FreeCashFlow']].copy().pct_change(axis=0)*100).fillna(0.0)
            trend = df.mean(axis=1)
            
            fig = go.Figure()
            fig.add_trace(
                go.Bar(
                    x=df.index,
                    y=df['TotalRevenue'],
                    marker=dict(color='cadetblue',line=dict(color='#000',width=1)),
                    name='Revenue'
                )
            )
            fig.add_trace(
                go.Bar(
                    x=df.index,
                    y=df['FreeCashFlow'],
                    marker=dict(color='forestgreen',line=dict(color='#000',width=1)),
                    name='FCF'
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=trend.index,
                    y=trend,mode='lines+markers',
                    line=dict(color='peru',width=2),
                    marker=dict(color='peru',size=10,symbol='diamond',line=dict(color='#000',width=1)),
                    name='Trend'
                )
            )
            fig.update_layout(
                paper_bgcolor='#000',
                plot_bgcolor='#000',
                xaxis=dict(gridcolor='#bbb'),
                yaxis=dict(gridcolor='#222',title='Change(%)'),
                font=dict(color='#fff'),
                title=f'{stock} {self.t} Growth'
            )
            fig.show()
            
    def plot_diversification(self):
        if self.state == 0:
            return
        self.get_industry()
        fig = go.Figure()
        values = []
        
        for sector, stocks in self.sectors.groupby(self.sectors.values):
            values.append({
                'industry':sector,
                'data':{'nstocks':len(stocks),'stocks':[stock.strip() for stock in stocks.index]}
            })
        cm = px.colors.sequential.Rainbow
        fig.add_trace(
            go.Pie(
                labels=[i['industry'] for i in values],
                values=[i['data']['nstocks'] for i in values],
                name='Stocks',
                customdata=[i['data']['stocks'] for i in values],
                hovertemplate='%{customdata}',
                textinfo='none',
                marker=dict(colors=cm,line=dict(color='#000',width=1)),
                hole=0.6,
                
            )
        )
        fig.update_layout(
                paper_bgcolor='#000',
                plot_bgcolor='#000',
                font=dict(color='#fff'),
                title='Diversification'
            )
        fig.show()
    
    def print_statistics(self):
        if self.state == 0:
            return
        stats_df = pd.DataFrame(columns=['cagr','mean','std','trend','sr'])
        
        for stock in self.stocks:
            df = pd.read_csv(f'{stock}_financials-{self.t}.csv',index_col=0,parse_dates=True)
            
            mean = (df['NetIncome'].pct_change()*100).dropna().mean()
            std = (df['NetIncome'].pct_change()*100).dropna().std()
            trend = (df[['TotalRevenue','FreeCashFlow']].pct_change(axis=0)*100).mean(axis=1).mean()
            
            if df['NetIncome'].iloc[-1] <= 0 or df['NetIncome'].iloc[0] <= 0:
                cagr = np.NaN
                sr = np.NaN
            else:
                cagr = (df['NetIncome'].iloc[-1]/df['NetIncome'].iloc[0]-1)*100
                sr = (0.7*cagr+0.3*mean)/std
            
            stats_df.loc[stock,'cagr'] = round(cagr,2)
            stats_df.loc[stock,'mean'] = round(mean,2)
            stats_df.loc[stock,'std'] = round(std,2)
            stats_df.loc[stock,'trend'] = round(trend,2)
            stats_df.loc[stock,'sr'] = round(sr,2)
            
        stats_df.sort_values(by='sr',ascending=False,inplace=True)
        print(f'\n{stats_df}')


if __name__ == '__main__': 
    si = StockInfo('WL.txt','q')
    si.print_statistics()
    si.plot_diversification()